# Derivation of an efficient implementation of the Jaumann stress rate

In [1]:
import sympy

In [47]:
l11, l12, l13, l21, l22, l23, l31, l32, l33 = sympy.symbols("l_11, l_12, l_13, l_21, l_22, l_23, l_31, l_32, l_33")
d11, d12, d13, d21, d22, d23, d31, d32, d33 = sympy.symbols("d_11, d_12, d_13, d_21, d_22, d_23, d_31, d_32, d_33")
s11, s12, s13, s21, s22, s23, s31, s32, s33 = sympy.symbols("\sigma_11, \sigma_12, \sigma_13, \sigma_21, \sigma_22, \sigma_23, \sigma_31, \sigma_32, \sigma_33")
sm1, sm2, sm3, sm4, sm5, sm6 = sympy.symbols("\sigma_1^m, \sigma_2^m, \sigma_3^m, \sigma_4^m, \sigma_5^m, \sigma_6^m")
del_t = sympy.symbols("\Delta t")
L = sympy.Matrix([[l11, l12, l13], [l21, l22, l23], [l31, l32, l33]])
sigma = sympy.Matrix([[s11, s12, s13],[s12, s22, s23],[s13, s23, s33]])
sigma_mandel = sympy.Matrix([sm1,sm2,sm3,sm4,sm5,sm6])

In [10]:
L

Matrix([
[l_11, l_12, l_13],
[l_21, l_22, l_23],
[l_31, l_32, l_33]])

In [11]:
sigma

Matrix([
[\sigma_11, \sigma_12, \sigma_13],
[\sigma_12, \sigma_22, \sigma_23],
[\sigma_13, \sigma_23, \sigma_33]])

In [48]:
sigma_mandel

Matrix([
[\sigma_1^m],
[\sigma_2^m],
[\sigma_3^m],
[\sigma_4^m],
[\sigma_5^m],
[\sigma_6^m]])

In [17]:
W = (L-L.T)/2
W

Matrix([
[               0,  l_12/2 - l_21/2, l_13/2 - l_31/2],
[-l_12/2 + l_21/2,                0, l_23/2 - l_32/2],
[-l_13/2 + l_31/2, -l_23/2 + l_32/2,               0]])

In [19]:
D = (L+L.T)/2
D

Matrix([
[           l_11, l_12/2 + l_21/2, l_13/2 + l_31/2],
[l_12/2 + l_21/2,            l_22, l_23/2 + l_32/2],
[l_13/2 + l_31/2, l_23/2 + l_32/2,            l_33]])

In [52]:
def to_mandel(M):
    sqrt = sympy.sqrt(2)
    return sympy.Matrix([M[0,0], M[1,1], M[2,2], sqrt*M[1,2], sqrt*M[0,2], sqrt*M[0,1]])

def to_tensor(mandel):
    sqrt = 1/sympy.sqrt(2)
    return sympy.Matrix([
        [mandel[0], sqrt*mandel[5], sqrt*mandel[4]],
        [sqrt*mandel[5], mandel[1], sqrt*mandel[3]],
        [sqrt*mandel[4], sqrt*mandel[3], mandel[2]]
    ])

## Strain rate in Mandel form

$D=0.5\cdot(L+L^\top)$ is often referred to as the strain rate which we want to provide in Mandel form without calculating $D$ inthe first place:

In [34]:
sympy.simplify(to_mandel(D))

Matrix([
[                   l_11],
[                   l_22],
[                   l_33],
[sqrt(2)*(l_23 + l_32)/2],
[sqrt(2)*(l_13 + l_31)/2],
[sqrt(2)*(l_12 + l_21)/2]])

## Rotation of stresses

We want to apply the Jaumann stress rate on a Stress vector in Mandel form without constructing the full stress tensor and also maybe not constructing $W=0.5\cdot(L-L^\top)$

In [61]:
W_sigma= W * to_tensor(sigma_mandel)
del_sigma = to_mandel(W_sigma+W_sigma.T)
sympy.simplify(del_sigma)

Matrix([
[                                                                           sqrt(2)*(\sigma_5^m*(l_13 - l_31) + \sigma_6^m*(l_12 - l_21))/2],
[                                                                           sqrt(2)*(\sigma_4^m*(l_23 - l_32) - \sigma_6^m*(l_12 - l_21))/2],
[                                                                          sqrt(2)*(-\sigma_4^m*(l_23 - l_32) - \sigma_5^m*(l_13 - l_31))/2],
[sqrt(2)*(-2*\sigma_2^m*(l_23 - l_32) + 2*\sigma_3^m*(l_23 - l_32) - sqrt(2)*\sigma_5^m*(l_12 - l_21) - sqrt(2)*\sigma_6^m*(l_13 - l_31))/4],
[sqrt(2)*(-2*\sigma_1^m*(l_13 - l_31) + 2*\sigma_3^m*(l_13 - l_31) + sqrt(2)*\sigma_4^m*(l_12 - l_21) - sqrt(2)*\sigma_6^m*(l_23 - l_32))/4],
[sqrt(2)*(-2*\sigma_1^m*(l_12 - l_21) + 2*\sigma_2^m*(l_12 - l_21) + sqrt(2)*\sigma_4^m*(l_13 - l_31) + sqrt(2)*\sigma_5^m*(l_23 - l_32))/4]])

For a better view we multiply by $\sqrt 2$ which yields

In [63]:
sympy.simplify(del_sigma)*sympy.sqrt(2)

Matrix([
[                                                                           \sigma_5^m*(l_13 - l_31) + \sigma_6^m*(l_12 - l_21)],
[                                                                           \sigma_4^m*(l_23 - l_32) - \sigma_6^m*(l_12 - l_21)],
[                                                                          -\sigma_4^m*(l_23 - l_32) - \sigma_5^m*(l_13 - l_31)],
[-\sigma_2^m*(l_23 - l_32) + \sigma_3^m*(l_23 - l_32) - sqrt(2)*\sigma_5^m*(l_12 - l_21)/2 - sqrt(2)*\sigma_6^m*(l_13 - l_31)/2],
[-\sigma_1^m*(l_13 - l_31) + \sigma_3^m*(l_13 - l_31) + sqrt(2)*\sigma_4^m*(l_12 - l_21)/2 - sqrt(2)*\sigma_6^m*(l_23 - l_32)/2],
[-\sigma_1^m*(l_12 - l_21) + \sigma_2^m*(l_12 - l_21) + sqrt(2)*\sigma_4^m*(l_13 - l_31)/2 + sqrt(2)*\sigma_5^m*(l_23 - l_32)/2]])

Note, that from $L$, only the differences $[l_{12}-l_{21},l_{13}-l_{31}, l_{23}-l_{32}]$ are needed which should give an additional speedup compared to computing $W$ explicitely.